<h1> NYC Restaurants Analysis - Data Extraction & Preprocessing </h1>
    
In order to get data on NYC restaurants, we will use two main data sources: 
* **DOHMH New York City Restaurant Inspection Results:** a dataset containing all sanitary inspection results for NYC restaurants along with the restaurants features (e.g. cuisine type, longitude, latitude, phone number)
* **Yelp API:** it will enable us to retrieve a satisfaction score for a restaurant as well as some other characteristics (e.g., reviews, price category...)

We will start by cleaning the data from DOHMH to keep only the latest inspection results for each business and estimate the proportion of restaurants for which the grade is pending to know if it's reasonable to drop it.

Then, we will retrieve the data corresponding to the remaining restaurants using Yelp API phone search to get the business details and id. Once we will have the Yelp ID, we will be able to get an excerpt of the restaurant's reviews by using another API endpoint.

Finally, we will clean this data and save the resulting dataset in a csv file to be used later for EDA and modeling.

In [1]:
# Necessary imports
import pandas as pd
import numpy as np

<h1> Part 1 - Data Preprocessing - DOMHM </h1>

<h2> Step 0: Load DOMHM data</h2>

This data come from [this NYC open data page](https://data.cityofnewyork.us/Health/DOHMH-New-York-City-Restaurant-Inspection-Results/43nn-pn8j) and is updated everyday. The data used in this notebook corresponds to the csv file from Jan 19, 2024.

In [2]:
raw_data = pd.read_csv("DOHMH_New_York_City_Restaurant_Inspection_Results_20240119.csv")

In [3]:
# Visualize the first 10 rows
raw_data.head(10)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,50119424,ARS NOVA THEATER,Manhattan,27,BARROW STREET,10014.0,2124899800,NaN,01/01/1900,NaN,...,NaN,40.732493,-74.003078,102.0,3.0,6700.0,1010159.0,1.005900e+09,MN23,NaN
1,50135094,BARO BY CHEFS SOCIETY,Manhattan,23,WEST 31 STREET,10001.0,2013880062,NaN,01/01/1900,NaN,...,NaN,40.747007,-73.987054,105.0,3.0,7600.0,1015810.0,1.008330e+09,MN17,NaN
2,50139189,ACKEE BY THE SALTFISH,Queens,2426,47TH ST,11103.0,5165036754,NaN,01/01/1900,NaN,...,NaN,40.766568,-73.905124,401.0,22.0,14100.0,4013583.0,4.007320e+09,QN70,NaN
3,50146322,NaN,Brooklyn,2138,CATON AVENUE,11226.0,3479936113,NaN,01/01/1900,NaN,...,NaN,40.652931,-73.958970,314.0,40.0,79602.0,3116601.0,3.050830e+09,BK60,NaN
4,50105157,GOLD STAR BEER COUNTER,Brooklyn,340,STERLING PLACE,11238.0,7736776855,NaN,01/01/1900,NaN,...,NaN,40.675339,-73.966865,308.0,35.0,20700.0,3029340.0,3.011710e+09,BK64,NaN
5,50134849,DELTA SKY CLUB,Queens,4,Terminal Concourse A JFK International,11430.0,4047142047,NaN,01/01/1900,NaN,...,NaN,40.648313,-73.788281,483.0,31.0,71600.0,4000000.0,4.142600e+09,QN98,NaN
6,50143189,NaN,Manhattan,1389,MADISON AVENUE,10029.0,2129871700,NaN,01/01/1900,NaN,...,NaN,40.787615,-73.953867,111.0,8.0,16002.0,1051443.0,1.016020e+09,MN33,NaN
7,50144187,LA BARRA,Manhattan,501,WEST 30 STREET,10001.0,6464951242,NaN,01/01/1900,NaN,...,NaN,40.752132,-74.001148,104.0,3.0,9900.0,1089323.0,1.007020e+09,MN13,NaN
8,50129499,BK WILD,Brooklyn,103,NORTH 3 STREET,11249.0,2124959672,Vegan,12/05/2022,No violations were recorded at the time of thi...,...,Pre-permit (Non-operational) / Initial Inspection,40.716792,-73.962076,301.0,34.0,55500.0,3335727.0,3.023500e+09,BK73,NaN
9,50146157,NaN,Manhattan,342,7 AVENUE,10001.0,6464760498,NaN,01/01/1900,NaN,...,NaN,40.747987,-73.992818,105.0,3.0,9500.0,1014289.0,1.007790e+09,MN17,NaN


Just from looking at the first 10 rows, we can already tell that data cleaning will be necessary as only the 9th row has a non-empty cuisine description and inspection results.

This is likely to happen because, as written on the NYC Open Data website:
*"Records are also included for each restaurant that has applied for a permit but has not yet been inspected and for inspections resulting in no violations. Establishments with inspection date of 1/1/1900 are new establishments that have not yet received an inspection."*

In [4]:
# Let's look at our dataset characteristics
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212592 entries, 0 to 212591
Data columns (total 27 columns):
 #   Column                 Non-Null Count   Dtype  
---  ------                 --------------   -----  
 0   CAMIS                  212592 non-null  int64  
 1   DBA                    211856 non-null  object 
 2   BORO                   212592 non-null  object 
 3   BUILDING               212275 non-null  object 
 4   STREET                 212589 non-null  object 
 5   ZIPCODE                209902 non-null  float64
 6   PHONE                  212590 non-null  object 
 7   CUISINE DESCRIPTION    210030 non-null  object 
 8   INSPECTION DATE        212592 non-null  object 
 9   ACTION                 210030 non-null  object 
 10  VIOLATION CODE         208906 non-null  object 
 11  VIOLATION DESCRIPTION  208906 non-null  object 
 12  CRITICAL FLAG          212592 non-null  object 
 13  SCORE                  202248 non-null  float64
 14  GRADE                  103598 non-nu

Apparently, the scarcest columns are GRADE DATE and GRADE (a bit less than 50% of the dataset has a value). We will need to investigate to understand if this is due to the grade being associated to only one row per restaurant. Indeed, in the raw data, the CAMIS isn't a primary key because a restaurant can have multiple violations, resulting in multiple rows.

Most other columns have less than 2% missing values so it's fine if we don't focus on them for now.

In [5]:
# Compute the number of unique restaurants
len(raw_data['CAMIS'].unique())

28600

There are "only" 28600 unique restaurants in our dataset. Let's take a quick look at the distribution of the rows between these CAMIS:

In [6]:
raw_data['CAMIS'].value_counts()

40398688    65
41406895    59
40365904    59
50089474    54
50123073    51
            ..
50139436     1
50129242     1
50138963     1
50143818     1
50124792     1
Name: CAMIS, Length: 28600, dtype: int64

The top 5 CAMIS with the most rows have more than 50 rows associated to them 🤯
Let's deep dive on some of them to understand what this corresponds to and whether we should exclude them or not:

In [7]:
raw_data[raw_data['CAMIS']==40398688].ACTION.value_counts()

Violations were cited in the following area(s).                                                                                       49
Establishment Closed by DOHMH. Violations were cited in the following area(s) and those requiring immediate action were addressed.    14
Establishment re-opened by DOHMH.                                                                                                      2
Name: ACTION, dtype: int64

In [8]:
raw_data[raw_data['CAMIS']==40398688].GRADE.value_counts()

A    12
C    10
Name: GRADE, dtype: int64

In [9]:
raw_data[raw_data['CAMIS']==40398688]['GRADE DATE'].value_counts()

10/11/2019    8
05/07/2019    4
05/08/2018    3
11/16/2018    3
06/02/2017    2
11/22/2017    2
Name: GRADE DATE, dtype: int64

In [10]:
raw_data[raw_data['CAMIS']==40398688][['GRADE','GRADE DATE']]

,GRADE,GRADE DATE
4770,A,05/08/2018
13836,C,06/02/2017
27148,NaN,NaN
31629,NaN,NaN
32979,NaN,NaN
...,...,...
198205,NaN,NaN
203395,NaN,NaN
203550,NaN,NaN
208183,NaN,NaN


From what we see, this restaurant has had a lot of violations in the past and has known a succession of closures/openings. According to the last inspection date, the data isn't very recent. 

We will probably want to filter on the grade date later to only keep results from inspections held after Jan 1, 2022. 

<h2> Step 1: Drop the restaurants that have not yet received an inspection </h2>

As explained on NYC Open Data, the restaurants with an inspection date of 1/1/1900 are new establishments that have not yet received an inspection. Therefore, our next step is to get rid of these restaurants.

In [11]:
raw_data_inspection = raw_data[raw_data['INSPECTION DATE']!='01/01/1900']

In [12]:
raw_data_inspection.head(10)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
8,50129499,BK WILD,Brooklyn,103,NORTH 3 STREET,11249.0,2124959672,Vegan,12/05/2022,No violations were recorded at the time of thi...,...,Pre-permit (Non-operational) / Initial Inspection,40.716792,-73.962076,301.0,34.0,55500.0,3335727.0,3.023500e+09,BK73,NaN
12,50112714,GENESIS RESTAURANT,Queens,5627,58TH ST,11378.0,9293375300,Latin American,10/28/2021,Violations were cited in the following area(s).,...,Pre-permit (Operational) / Initial Inspection,40.724601,-73.912450,402.0,26.0,21900.0,4059724.0,4.026640e+09,QN31,NaN
18,50110709,MAMAN,Manhattan,1424,3 AVENUE,10028.0,6464767660,Coffee/Tea,08/23/2022,No violations were recorded at the time of thi...,...,Cycle Inspection / Re-inspection,40.775413,-73.956478,108.0,5.0,14000.0,1047526.0,1.015090e+09,MN40,NaN
19,50006337,D ELICI,Manhattan,71,NASSAU STREET,10038.0,2125875740,American,05/24/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.709632,-74.008325,101.0,1.0,1502.0,1001215.0,1.000798e+09,MN25,NaN
30,50079292,CANAS RESTAURANT AND BAR,Queens,14407,JAMAICA AVE,11435.0,9292447267,Latin American,05/04/2023,No violations were recorded at the time of thi...,...,Administrative Miscellaneous / Initial Inspection,40.702217,-73.811210,412.0,24.0,24000.0,4534220.0,4.096740e+09,QN61,NaN
44,50002435,CIBO MARKET,Queens,NaN,JFK INTERNATIONAL AIRPORT,11430.0,9292591541,Sandwiches/Salads/Mixed Buffet,12/07/2016,No violations were recorded at the time of thi...,...,Cycle Inspection / Initial Inspection,40.648313,-73.788281,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,50007093,PHILIPPE NYC I,Manhattan,33,EAST 60 STREET,10022.0,2128715603,Chinese,03/31/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.764095,-73.970666,108.0,4.0,11401.0,1078238.0,1.013750e+09,MN40,NaN
59,41646967,FAY DA BAKERY,Manhattan,321,6 AVENUE,10014.0,2122559888,Chinese,05/03/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.730989,-74.001461,102.0,3.0,6700.0,1010073.0,1.005890e+09,MN23,NaN
61,50100995,DAIKANYAMA,Manhattan,1000,THIRD AVENUE,10022.0,2128371853,Japanese,06/22/2023,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.761887,-73.966349,108.0,4.0,11402.0,1041904.0,1.013940e+09,MN40,NaN
62,50110743,AJI FUSION SUSHI,Brooklyn,201,5 AVENUE,11217.0,7188772466,Asian/Asian Fusion,07/07/2022,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.676594,-73.980319,306.0,39.0,13100.0,3019762.0,3.009530e+09,BK37,NaN


In [13]:
raw_data_inspection.shape[0]

210030

In [14]:
len(raw_data_inspection.CAMIS.unique())

26038

We only lose about 7% of the restaurants compared to our initial dataset, which is pretty decent but we need to do more cleaning to only keep the restaurants with a grade, and take the latest one.

<h2> Step 2: Restrict our dataset to the latest inspection results </h2>

\
First, we will drop all the rows that don't correspond to the latest inspection date with non-null grade. 

Then, we will restrict our dataset to the inspection results after Jan 1, 2022.



<h3> 2.1. Drop the rows with a missing grade </h3>

In [15]:
# We want to drop the rows for which no grade is available
has_grade = raw_data_inspection['GRADE'].isna() == False
raw_data_grade_only = raw_data_inspection[has_grade]

In [16]:
len(raw_data_grade_only.CAMIS.unique())

26038

In [17]:
len(raw_data_grade_only)

103598

As expected, after dropping the rows with a missing grade, we keep 103,598 rows corresponding to 26,038 unique restaurants.

<h3> 2.2. Select only the latest inspection date for each CAMIS </h3>

Now, let's select only the rows that correspond to the latest inspection date!

In [18]:
data_latest_grade = raw_data_grade_only.copy()

In [19]:
# Convert the INSPECTION DATE column to datetime
data_latest_grade['INSPECTION DATE'] = pd.to_datetime(raw_data_grade_only['INSPECTION DATE'])

In [20]:
# Sort the copied dataframe by CAMIS and INSPECTION DATE in descending order
data_latest_grade.sort_values(['CAMIS', 'INSPECTION DATE'], ascending=[True, False], inplace=True)

# Keep only the rows with the latest INSPECTION DATE for each CAMIS
df_latest_grade = data_latest_grade.groupby('CAMIS').head(1)

# Reset index
df_latest_grade.reset_index(drop=True, inplace=True)

In [21]:
len(df_latest_grade)

26038

In [22]:
df_latest_grade.GRADE.value_counts()

A    20140
N     2122
B     1984
Z      980
C      811
P        1
Name: GRADE, dtype: int64

<h3> 2.3. Keep only the grades that are meaningful to our analysis </h3>

**Excerpt from the Health Department documentation:**

*Violations found during inspections carry point values, and a restaurant’s score corresponds to a letter grade. The point/grade cut-offs are the same as for mobile food vending letter grading, with fewer points corresponding to a better grade:*

* *"A" grade: 0 to 13 points for sanitary violations*
* *"B" grade: 14 to 27 points for sanitary violations*
* *"C" grade: 28 or more points for sanitary violations*

*"Grade Pending" means that on both the initial inspection and reinspection, the restaurant received 14 or more points. Following the reinspection, the restaurant can post "Grade Pending" or the letter grade while they have the opportunity for an administrative hearing to determine the final grade.*

Besides, in our data, N = Not Yet Graded, Z = Grade Pending and P = Grade Pending issued on re-opening following an initial inspection that resulted in a closure.

It'd thus make sense to drop the rows with a N as it is a very neutral grade. Besides, as P only corresponds to one restaurant, we will drop it too.

In [23]:
# Filter to keep only the A, B, C and Z grades
has_defined_grade = df_latest_grade['GRADE'].isin(['A','B','C','Z'])
df_latest_defined_grade = df_latest_grade[has_defined_grade]

In [24]:
len(df_latest_defined_grade)

23915

In [25]:
df_latest_defined_grade.head(3)

,CAMIS,DBA,BORO,BUILDING,STREET,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,...,INSPECTION TYPE,Latitude,Longitude,Community Board,Council District,Census Tract,BIN,BBL,NTA,Location Point1
0,30075445,MORRIS PARK BAKE SHOP,Bronx,1007,MORRIS PARK AVENUE,10462.0,7188924968,Bakery Products/Desserts,2023-08-22,Violations were cited in the following area(s).,...,Cycle Inspection / Re-inspection,40.848231,-73.855972,211.0,13.0,25200.0,2045445.0,2.041270e+09,BX37,NaN
1,30112340,WENDY'S,Brooklyn,469,FLATBUSH AVENUE,11225.0,7182875005,Hamburgers,2023-07-21,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.662652,-73.962081,309.0,40.0,32700.0,3029737.0,3.011970e+09,BK60,NaN
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,351,WEST 57 STREET,10019.0,2122452912,Irish,2023-04-23,Violations were cited in the following area(s).,...,Cycle Inspection / Initial Inspection,40.767326,-73.984310,104.0,3.0,13900.0,1026048.0,1.010480e+09,MN15,NaN


<h3> 2.4. Restrict to only the inspections after Jan 1, 2022 </h3>
    
We will now filter on the latest inspection date to make sure that we only consider inspections after Jan 1, 2022.

In [26]:
is_in_2022_or_after = df_latest_defined_grade['INSPECTION DATE']>='2022-01-01'
df_recent_grade = df_latest_defined_grade[is_in_2022_or_after]

In [27]:
len(df_recent_grade)

22871

Approximately 80% of the restaurants included in the original dataset satisfy our criteria on grade. 

<h2> Step 3: Drop all rows with a NaN value in some column of interest </h2>

<h3> 3.1. Select only the columns we are interested in </h3>

There are some columns we're not really interested in such as *Community Board* or *BIN*. Let's list the columns we want to keep: CAMIS, DBA, BORO, ZIPCODE, PHONE, CUISINE DESCRIPTION, INSPECTION DATE, ACTION, GRADE, SCORE, Latitude, Longitude.

In [28]:
unique_restaurants_refined = df_recent_grade[['CAMIS', 'DBA', 'BORO', 'ZIPCODE', 'PHONE', 'CUISINE DESCRIPTION', 'INSPECTION DATE', 'ACTION', 'GRADE', 'SCORE', 'Latitude', 'Longitude']]

In [29]:
unique_restaurants_refined.head(10)

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude
0,30075445,MORRIS PARK BAKE SHOP,Bronx,10462.0,7188924968,Bakery Products/Desserts,2023-08-22,Violations were cited in the following area(s).,A,12.0,40.848231,-73.855972
1,30112340,WENDY'S,Brooklyn,11225.0,7182875005,Hamburgers,2023-07-21,Violations were cited in the following area(s).,A,9.0,40.662652,-73.962081
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,10019.0,2122452912,Irish,2023-04-23,Violations were cited in the following area(s).,A,10.0,40.767326,-73.984310
3,40356018,RIVIERA CATERERS,Brooklyn,11224.0,7183723031,American,2022-02-01,Violations were cited in the following area(s).,A,7.0,40.579896,-73.982087
4,40356483,WILKEN'S FINE FOOD,Brooklyn,11234.0,7184443838,Sandwiches,2022-08-19,Violations were cited in the following area(s).,A,2.0,40.620112,-73.906989
5,40356731,TASTE THE TROPICS ICE CREAM,Brooklyn,11226.0,7188560821,Frozen Desserts,2023-01-17,Violations were cited in the following area(s).,A,9.0,40.640795,-73.948488
7,40359480,1 EAST 66TH STREET KITCHEN,Manhattan,10065.0,2128793900,American,2022-05-03,Violations were cited in the following area(s).,A,9.0,40.768547,-73.969581
8,40359705,NATHAN'S FAMOUS,Brooklyn,11224.0,7183332202,Hotdogs,2023-04-26,Violations were cited in the following area(s).,A,13.0,40.575537,-73.981652
9,40360045,SEUDA FOODS,Brooklyn,11223.0,7183751500,Jewish/Kosher,2023-01-31,Violations were cited in the following area(s).,A,13.0,40.606187,-73.965466
10,40361618,SAL'S DELI,Queens,11356.0,7186619498,Sandwiches,2023-04-26,Violations were cited in the following area(s).,A,13.0,40.781674,-73.839416


In [30]:
unique_restaurants_refined.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22871 entries, 0 to 26035
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CAMIS                22871 non-null  int64         
 1   DBA                  22871 non-null  object        
 2   BORO                 22871 non-null  object        
 3   ZIPCODE              22605 non-null  float64       
 4   PHONE                22871 non-null  object        
 5   CUISINE DESCRIPTION  22871 non-null  object        
 6   INSPECTION DATE      22871 non-null  datetime64[ns]
 7   ACTION               22871 non-null  object        
 8   GRADE                22871 non-null  object        
 9   SCORE                22871 non-null  float64       
 10  Latitude             22827 non-null  float64       
 11  Longitude            22827 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 2.3+ MB


<h3> 3.2. Drop all rows with at least 1 NaN value </h3>

In [31]:
unique_restaurants_with_info = unique_restaurants_refined.dropna()

In [32]:
unique_restaurants_with_info.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22561 entries, 0 to 26035
Data columns (total 12 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   CAMIS                22561 non-null  int64         
 1   DBA                  22561 non-null  object        
 2   BORO                 22561 non-null  object        
 3   ZIPCODE              22561 non-null  float64       
 4   PHONE                22561 non-null  object        
 5   CUISINE DESCRIPTION  22561 non-null  object        
 6   INSPECTION DATE      22561 non-null  datetime64[ns]
 7   ACTION               22561 non-null  object        
 8   GRADE                22561 non-null  object        
 9   SCORE                22561 non-null  float64       
 10  Latitude             22561 non-null  float64       
 11  Longitude            22561 non-null  float64       
dtypes: datetime64[ns](1), float64(4), int64(1), object(6)
memory usage: 2.2+ MB


After this cleaning step, we end up with **22561 unique NYC restaurants**, their location, phone number, cuisine and sanitary characteristics. 

In order to use this data as a starting point to query Yelp API and obtain more restaurants attributes, we will need to **reformat the phone number**. 

Besides, we will **save this dataset as a csv file** to avoid running the part 1 of the notebook again.

<h2> Step 4: Format the phone number for Yelp's API and export our dataset as a csv file </h2>

<h3> 4.1. Reformat the PHONE column to be able to easily query Yelp API </h3>

In [33]:
unique_restaurants_phone_formatted = unique_restaurants_with_info.copy()
unique_restaurants_phone_formatted['PHONE'] = unique_restaurants_with_info['PHONE'].apply(lambda x: '+1'+x)

In [34]:
# Let's see how many phone duplicates we have
unique_restaurants_phone_formatted.shape[0]-len(unique_restaurants_phone_formatted['PHONE'].unique())

1446

We choose to remove these duplicates as we will use the phone number to query Yelp API and we don't wanna take the risk to associate the wrong ratings to a restaurant.

<h3> 4.2. Remove restaurants that have a non-unique phone number </h3>

In [35]:
# We decide to only drop the duplicates based on the phone number so far
unique_restaurants_formatted = unique_restaurants_phone_formatted.drop_duplicates(subset=['PHONE'], keep=False)

In [36]:
# How many restaurants do we have left?
unique_restaurants_formatted.shape[0]

20323

Eventually, we keep **20,323 restaurants in our dataset**, which is pretty good compared to the 28k initial restaurants and the 24k restaurants with a grade.

<h3> 4.3. Export the clean dataset as a csv file for future operations </h3

In [43]:
# We will drop the index as it is useless for our analysis and hasn't been reset for a while
unique_restaurants_formatted.to_csv('unique_restaurants_formatted.csv',index=False)

<h1> Part 2 - Querying Yelp API to enrich the DOMHM data </h1> 

<h2> Step 0: Create enough API keys to handle Yelp API daily credits limit </h2>

\
For this project, we'll use **Yelp Fusion API** to find information on the restaurants in NYC! If you don't have a Yelp account yet, you can go to this [web page](https://docs.developer.yelp.com/docs/fusion-intro) and create one.

Once you have created your account, you can create an API key:
1. Go to [Create App](https://www.yelp.com/developers/v3/manage_app)
2. In the create new app form, enter information about your app, then agree to Yelp API Terms of Use and Display Requirements. Then click the Submit button.
3. You will now have an API Key.

> 💡 As we have more than 20k restaurants to look for in Yelp and the daily credit limit per account is 5k, we will need to create a few API keys and use them sequentially.

In [80]:
# Yelp Credentials
# After running the notebook, I intentionally deleted mine for security concerns so you'll need to create your own

# API KEY 1
CLIENT_ID = ""
API_KEY = ""

# API KEY 2
#CLIENT_ID = ""
#API_KEY = ""

# API KEY 3
#CLIENT_ID = ""
#API_KEY = ""

# API KEY 4
#CLIENT_ID = ""
#API_KEY = ""

# API KEY 5
#CLIENT_ID = ""
#API_KEY = ""

In [51]:
# API constants
API_HOST = 'https://api.yelp.com/v3'     # The API url
BUSINESS_SEARCH = '/businesses/search' # The path for a more generic request using keywords or latitude/longitude
PHONE_SEARCH_PATH = '/businesses/search/phone' # The path for a request using the phone number of the business
BUSINESS_PATH = '/businesses/'

<h2> Step 1: Download the formatted restaurant table from NYC Open Data DOMHM </h2>

In [39]:
restaurants_table = pd.read_csv('unique_restaurants_formatted.csv')

In [40]:
restaurants_table.head()

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude
0,30075445,MORRIS PARK BAKE SHOP,Bronx,10462.0,+17188924968,Bakery Products/Desserts,2023-08-22,Violations were cited in the following area(s).,A,12.0,40.848231,-73.855972
1,30112340,WENDY'S,Brooklyn,11225.0,+17182875005,Hamburgers,2023-07-21,Violations were cited in the following area(s).,A,9.0,40.662652,-73.962081
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,10019.0,+12122452912,Irish,2023-04-23,Violations were cited in the following area(s).,A,10.0,40.767326,-73.984310
3,40356018,RIVIERA CATERERS,Brooklyn,11224.0,+17183723031,American,2022-02-01,Violations were cited in the following area(s).,A,7.0,40.579896,-73.982087
4,40356483,WILKEN'S FINE FOOD,Brooklyn,11234.0,+17184443838,Sandwiches,2022-08-19,Violations were cited in the following area(s).,A,2.0,40.620112,-73.906989


Everything looks good! We can now split this table into 5 parts (as we have 5 API keys) to remain below the daily limit of API calls ✨

<h2> Step 2: Split the table into 5 to remain below the daily limit of API calls </h2>

In [41]:
part_1 = restaurants_table.iloc[:4500]
part_1

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude
0,30075445,MORRIS PARK BAKE SHOP,Bronx,10462.0,+17188924968,Bakery Products/Desserts,2023-08-22,Violations were cited in the following area(s).,A,12.0,40.848231,-73.855972
1,30112340,WENDY'S,Brooklyn,11225.0,+17182875005,Hamburgers,2023-07-21,Violations were cited in the following area(s).,A,9.0,40.662652,-73.962081
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,10019.0,+12122452912,Irish,2023-04-23,Violations were cited in the following area(s).,A,10.0,40.767326,-73.984310
3,40356018,RIVIERA CATERERS,Brooklyn,11224.0,+17183723031,American,2022-02-01,Violations were cited in the following area(s).,A,7.0,40.579896,-73.982087
4,40356483,WILKEN'S FINE FOOD,Brooklyn,11234.0,+17184443838,Sandwiches,2022-08-19,Violations were cited in the following area(s).,A,2.0,40.620112,-73.906989
...,...,...,...,...,...,...,...,...,...,...,...,...
4495,41543356,LITTLE CAESARS,Queens,11429.0,+17182177271,Pizza,2023-05-22,Violations were cited in the following area(s).,A,5.0,40.713792,-73.737518
4496,41543410,"RED ROOSTER HARLEM, GINNY'S SUPPER CLUB",Manhattan,10027.0,+12127929001,American,2023-02-06,Violations were cited in the following area(s).,A,7.0,40.808158,-73.945186
4497,41543722,DOMINO'S,Manhattan,10033.0,+12127813700,Pizza,2022-09-29,Violations were cited in the following area(s).,A,13.0,40.850676,-73.937528
4498,41543738,NEW DYNASTY,Manhattan,10001.0,+12125949734,Chinese,2023-02-22,Violations were cited in the following area(s).,B,24.0,40.749384,-73.995514


In [42]:
part_2 = restaurants_table.iloc[4500:9000]
part_2

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude
4500,41543826,UMBERTO'S CLAM HOUSE,Manhattan,10013.0,+12124317545,Italian,2022-04-27,Violations were cited in the following area(s).,A,13.0,40.718385,-73.997774
4501,41543866,DALLAS BBQ,Queens,11374.0,+17185929000,Barbecue,2023-06-21,Violations were cited in the following area(s).,A,11.0,40.733428,-73.864170
4502,41543887,BEAUTY AND ESSEX,Manhattan,10002.0,+12126140146,American,2022-11-22,Violations were cited in the following area(s).,A,10.0,40.720454,-73.987208
4503,41543951,MABLE'S SMOKEHOUSE AND BANQUET HALL,Brooklyn,11249.0,+17182186655,American,2023-05-18,Violations were cited in the following area(s).,A,12.0,40.721036,-73.956431
4504,41544542,PRET A MANGER,Manhattan,10016.0,+12124018686,Soups/Salads/Sandwiches,2023-04-14,Violations were cited in the following area(s).,A,7.0,40.749806,-73.983546
...,...,...,...,...,...,...,...,...,...,...,...,...
8995,50048250,LOTTE NEW YORK PALACE -POMME PALAIS,Manhattan,10022.0,+12123037755,French,2022-12-06,Violations were cited in the following area(s).,A,2.0,40.758107,-73.975393
8996,50048254,POKE BOWL STATION,Brooklyn,11220.0,+13476714116,Hawaiian,2022-01-18,Violations were cited in the following area(s).,A,12.0,40.636093,-74.012524
8997,50048257,PATE PALO,Manhattan,10034.0,+16469186242,Latin American,2022-05-26,Violations were cited in the following area(s).,A,3.0,40.866428,-73.928333
8998,50048260,ZOUJI BBQ,Queens,11355.0,+17183530086,Barbecue,2023-01-19,Violations were cited in the following area(s).,C,39.0,40.752993,-73.827178


In [43]:
part_3 = restaurants_table.iloc[9000:13500]
part_3

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude
9000,50048274,CHINA PANDA,Brooklyn,11207.0,+17186491622,Chinese,2023-05-15,Violations were cited in the following area(s).,A,7.0,40.661472,-73.893931
9001,50048295,"DUNKIN', 'BASKIN ROBBINS",Brooklyn,11233.0,+17183425133,Donuts,2023-04-28,Violations were cited in the following area(s).,A,3.0,40.676639,-73.916327
9002,50048297,CAFE GOURMET,Brooklyn,11201.0,+17182220090,American,2023-02-02,Violations were cited in the following area(s).,A,7.0,40.689064,-73.982284
9003,50048298,"DUNKIN', 'BASKIN ROBBINS",Brooklyn,11208.0,+13474350286,Donuts,2023-06-07,Violations were cited in the following area(s).,A,9.0,40.683423,-73.872991
9004,50048300,DYNASTY,Queens,11354.0,+17189615381,Chinese,2023-06-15,Violations were cited in the following area(s).,A,11.0,40.765159,-73.819092
...,...,...,...,...,...,...,...,...,...,...,...,...
13495,50094458,DELICACY PASSION PATISSERIE,Brooklyn,11214.0,+17182662829,Bakery Products/Desserts,2023-11-28,Violations were cited in the following area(s).,A,13.0,40.599489,-73.989780
13496,50094460,DANNY'S CAFE CORP,Queens,11421.0,+19177447521,American,2023-05-24,Violations were cited in the following area(s).,A,10.0,40.693978,-73.851308
13497,50094476,GENTLE JUICE BAR,Brooklyn,11236.0,+16462586198,"Juice, Smoothies, Fruit Salads",2023-01-05,Violations were cited in the following area(s).,A,11.0,40.637308,-73.910468
13498,50094482,LA TRATTORIA,Manhattan,10017.0,+12126825656,Pizza,2023-01-13,Violations were cited in the following area(s).,C,29.0,40.751642,-73.970862


In [44]:
part_4 = restaurants_table.iloc[13500:18000]
part_4

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude
13500,50094519,WALK UP BAR -TIME OUT BAR,Brooklyn,11201.0,+19178105281,Other,2023-03-11,Violations were cited in the following area(s).,A,12.0,40.703294,-73.992047
13501,50094524,SUBWAY,Queens,11377.0,+16469341424,Sandwiches,2023-08-04,Violations were cited in the following area(s).,A,7.0,40.743531,-73.914550
13502,50094531,AYADA,Manhattan,10011.0,+12126459449,Thai,2023-05-31,Violations were cited in the following area(s).,A,13.0,40.741869,-74.004713
13503,50094533,AMICI RISTORANTE,Manhattan,10013.0,+13474696696,Italian,2023-04-17,Violations were cited in the following area(s).,A,10.0,40.719829,-73.997110
13504,50094543,MENG BAO LAI,Brooklyn,11232.0,+16469162888,Chicken,2022-06-10,Establishment re-opened by DOHMH.,Z,4.0,40.644916,-73.999777
...,...,...,...,...,...,...,...,...,...,...,...,...
17995,50124301,YUMMY JUICE BAR,Bronx,10462.0,+13472936151,"Juice, Smoothies, Fruit Salads",2023-01-03,Violations were cited in the following area(s).,A,12.0,40.854471,-73.866174
17996,50124303,LA CALENITA,Queens,11368.0,+19294606910,Latin American,2022-08-12,Violations were cited in the following area(s).,A,7.0,40.746468,-73.856586
17997,50124306,MIGHTY QUINN'S BBQ,Brooklyn,11215.0,+17188854367,Barbecue,2023-02-27,Violations were cited in the following area(s).,A,13.0,40.672305,-73.990299
17998,50124310,ASK SUSHI,Queens,11378.0,+13479249033,Japanese,2023-05-08,Violations were cited in the following area(s).,A,5.0,40.733094,-73.895916


In [45]:
part_5 = restaurants_table.iloc[18000:]
part_5

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude
18000,50124317,BAR 135,Manhattan,10013.0,+19172043512,American,2022-10-12,Violations were cited in the following area(s).,A,11.0,40.716378,-74.009660
18001,50124329,DI FARA,Manhattan,10038.0,+19173883130,Pizza,2023-08-09,Violations were cited in the following area(s).,Z,18.0,40.706829,-74.002121
18002,50124335,STARBUCKS,Manhattan,10004.0,+12127422490,Coffee/Tea,2022-10-18,Violations were cited in the following area(s).,A,8.0,40.705182,-74.011506
18003,50124336,STARBUCKS COFFEE #23044,Manhattan,10001.0,+16465993512,Coffee/Tea,2023-06-27,No violations were recorded at the time of thi...,A,0.0,40.749604,-73.991620
18004,50124353,FISH NOODLE HOUSE,Staten Island,10306.0,+19173928786,Chinese,2023-10-18,Violations were cited in the following area(s).,A,13.0,40.576186,-74.103990
...,...,...,...,...,...,...,...,...,...,...,...,...
20318,50144580,THE RAIL BAR & GRILL,Queens,11414.0,+19177691824,American,2023-12-07,Violations were cited in the following area(s).,A,11.0,40.660397,-73.830561
20319,50144643,CHOCOLATE HOUSE,Bronx,10451.0,+13477021996,Frozen Desserts,2024-01-04,Violations were cited in the following area(s).,A,11.0,40.827332,-73.924393
20320,50144844,TWO C CAKE,Brooklyn,11214.0,+19173466179,Coffee/Tea,2024-01-02,Violations were cited in the following area(s).,A,13.0,40.602931,-74.004386
20321,50145191,MAYA'S CAFE MEXICAN CUISINE,Staten Island,10302.0,+13478673800,Mexican,2024-01-16,Violations were cited in the following area(s).,A,11.0,40.625130,-74.135681


<h2> Step 3: Search for restaurants by phone number </h2>

<h3> 3.1. Define useful functions to query Yelp API </h3>

In [46]:
# First, we create a function to query Yelp API specifying an API key and a phone number
def get_restaurants_by_phone(api_key, phone):
    import requests
    
    search_data = {   
        'phone': phone
    }
    
    url = API_HOST + PHONE_SEARCH_PATH
    
    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }
    
    response = requests.request('GET', url, headers=headers, params=search_data).json()  
    businesses = response.get('businesses')   
    return businesses

In [47]:
# Then, we create a function that uses the previous function to build a dataframe of the query responses from a given dataframe that has at least a PHONE column
def generate_restaurants_table(df):
    restaurants_list = []

    for p in df['PHONE']:
        resto_list = get_restaurants_by_phone(API_KEY, p)

        if resto_list:
            if len(resto_list) == 1:
                for r in resto_list:
                    try:
                        restaurants_list.append([r['id'], r['name'], r['rating'], r['price'], r['phone'], r['transactions'], r['review_count']])
                    except:
                        print('Failed retrieving one or more of the attributes')
                        continue
            else:
                print('There is more than one restaurant matched')
        else:
            print('resto_list is empty')

    restaurants_df = pd.DataFrame(restaurants_list, columns=['ID', 'NAME', 'RATING', 'PRICE', 'PHONE', 'TRANSACTIONS', 'REVIEW_COUNT'])
    restaurants_df = restaurants_df.reset_index(drop=True)

    return restaurants_df

<h3> 3.2. Retrieve Yelp results for each subdataset, being careful to change the API key between each step </h3>

For this part, we will comment and uncomment the API keys in the first code cell of part 2. We will do so progressively, every time we switch to a new subdataset. This is necessary to ensure that we remain under the 5k API calls daily limit.

**FIRST SUBSET:**

In [48]:
df_restaurant_1 = generate_restaurants_table(part_1)
df_restaurant_1

Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is 

There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more 

resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restauran

There is more than one restaurant matched
There is more than one restaurant matched
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty


There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
There is more than one restaurant matched
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
Failed retrieving one or more of the attributes
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched


Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
Failed retrieving one or more of the attributes
There is more than one restaurant matched
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_li

,ID,NAME,RATING,PRICE,PHONE,TRANSACTIONS,REVIEW_COUNT
0,LG9gS6YnT5NwootBWp7gPA,Morris Park Bake Shop,4.0,$$,+17188924968,[pickup],66
1,AMxMPBkWi20dn_1BRalahA,Wendy's,1.5,$,+17182875005,[delivery],61
2,oo77ZO5cj9M2NRdTqEgHvQ,D.J. Reynolds,3.5,$$,+12122452912,[delivery],129
3,Pe6MsH2DW0CXjvUtxUpI4A,Wilkens Fine Foods,3.5,$$,+17184443838,"[pickup, delivery]",64
4,Q18r5FKi2qkQhtKZqWR8ww,Taste the Tropics USA,4.5,$,+17188560821,"[pickup, delivery]",59
...,...,...,...,...,...,...,...
3166,HhXXUpn4HlSqgWudxIcitA,Rubby's Pizzeria,3.5,$,+12127573707,[delivery],17
3167,hT17fpGyG_UvsBE4QUeYsA,Little Caesars,2.5,$,+17182177271,[delivery],16
3168,fxGpXRxFUDzlU3Cyszu4uQ,Red Rooster Harlem,3.5,$$$,+12127929001,"[pickup, delivery]",3865
3169,Gis29CeO78xoaaoIL6BRKw,Domino's Pizza,2.5,$,+12127813700,[],76


In [59]:
# Save the restaurant details into CSV files
df_restaurant_1.to_csv('yelp_restaurant_formatted_1.csv',index=False)

**SECOND SUBSET:**

In [69]:
df_restaurant_2 = generate_restaurants_table(part_2)

resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Fa

There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
There 

resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_

resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the att

resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attribu

resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more 

There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the a

In [70]:
df_restaurant_2

,ID,NAME,RATING,PRICE,PHONE,TRANSACTIONS,REVIEW_COUNT
0,0zxGc7hit7mFSk2o1PvIdQ,Umberto's Clam House,3.0,$$$,+12124317545,"[delivery, restaurant_reservation]",549
1,GkxvCwfCFaFY4UDvAkZ8dQ,Dallas BBQ,3.0,$$,+17185929000,"[delivery, pickup]",626
2,pfmAcS-g6SiNG0KlLvrqnA,Beauty & Essex,4.0,$$$,+12126140146,"[delivery, pickup]",3896
3,nsAU6F2vmdnphmfIa_RuWg,Mable's Smokehouse & Banquet Hall,4.0,$$,+17182186655,"[delivery, pickup]",1025
4,1fVp9c5Vw04ig171a_juRw,U Like,4.0,$,+17182439096,"[pickup, delivery]",53
...,...,...,...,...,...,...,...
2669,IhGROM0nyXSpw2Z2oV-6aA,McDonald's,1.5,$,+17188987437,[delivery],63
2670,Aprs5McFDpJCKTxWEJMlag,Mediterraneo Pizza,3.5,$,+12129270282,"[delivery, pickup]",30
2671,MP8Ey2ezsDCSowR-1X6hKg,Dosa Royale,3.5,$$,+17185763800,"[pickup, delivery]",283
2672,wYgp-defqwJPhjC6Y_WKWg,BurgerFi,4.0,$$,+17188360836,"[pickup, delivery]",365


In [71]:
# Save the restaurant details into CSV files
df_restaurant_2.to_csv('yelp_restaurant_formatted_2.csv',index=False)

**THIRD SUBSET:** 

In [74]:
df_restaurant_3 = generate_restaurants_table(part_3)

resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retri

There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty


Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is 

resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
Failed

resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is em

resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
re

Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
There is more than one restaurant matched
There is more than one restaurant matched
There is more than one restauran

Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
resto_

Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attribute

resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one 

In [75]:
df_restaurant_3

,ID,NAME,RATING,PRICE,PHONE,TRANSACTIONS,REVIEW_COUNT
0,05MSMrudU5N7UgImzuRVtw,China Panda,5.0,$,+17186491622,[delivery],1
1,LbohfoklOxK7mSMm96_wLw,Dunkin',2.0,$,+13474350286,[delivery],8
2,FiKu5C03pBiAn5v5DEL9UQ,The Butcher's Daughter,3.5,$$,+19173882132,"[pickup, delivery]",375
3,hTrn_YVmi5Kk7eVjTkBk9A,McDonald's,2.0,$,+17182712032,[delivery],32
4,GLWltF2ol24L4ypz5eTMhw,Dragon Palace,1.5,$,+17187121300,[pickup],11
...,...,...,...,...,...,...,...
1947,TGWtZ3AnRZtftqnuueoSCg,Bravazo Restaurante,4.0,$$,+19296667676,"[pickup, delivery]",17
1948,Z6UJiyRrgix9uIrzDcCMUg,Spread Love and Juice,4.5,$$,+13473743044,"[delivery, pickup]",59
1949,UQD85Z7Hiq_1Q3Jl7Jx-HQ,Hampton Inn NY-JFK,3.0,$$,+17183227500,[],116
1950,TkUWzErO5TBKl1jQcoKTng,Russ Pizza,4.0,$,+17183839463,"[delivery, pickup]",110


In [76]:
# Save the restaurant details into CSV files
df_restaurant_3.to_csv('yelp_restaurant_formatted_3.csv',index=False)

**FOURTH SUBSET:**

In [79]:
df_restaurant_4 = generate_restaurants_table(part_4)

resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_li

resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
There is more than o

resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attribu

resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes


Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empt

...

resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrievin

resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or

There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_

resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the att

resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty


In [80]:
df_restaurant_4

,ID,NAME,RATING,PRICE,PHONE,TRANSACTIONS,REVIEW_COUNT
0,C6GOh4ebry81vuG_765VvA,Paris Baguette,4.0,$$,+15165950404,"[pickup, delivery]",81
1,I6optu9x0JAf8oT5lqJ8qw,The Blue Dog Cookhouse & Bar,4.0,$$,+12124590700,"[delivery, pickup]",1533
2,bJUDGmgdiicJHTmPtblFRg,Holy Schnitzel,3.5,$$,+17187614659,"[pickup, delivery]",93
3,kGnJoYkzL-6aMrvwgYQXAg,Fan Shun,3.5,$,+17188977438,[delivery],38
4,jNPYmziqQjIMP-1pOgSZdw,Kape't Torta,4.5,$,+17188990240,"[delivery, pickup]",83
...,...,...,...,...,...,...,...
903,rmcuPv8ldKO5A8jd9-Y9zg,French Toast Bakery,3.5,$,+17184409182,"[pickup, delivery]",84
904,c5rP7Em_FarV9GBzpl-yNQ,Starbucks Reserve,3.5,$$,+17183840152,[delivery],137
905,5UmAZm8hvNwbClnSBdnKJg,Sleepwalk,4.5,$$,+13479874898,[],15
906,KEgdfVEVVgJFnGFiLNPDzQ,Sofia Pizza Shoppe,4.5,$,+12128888816,"[pickup, delivery]",251


In [81]:
# Save the restaurant details into CSV files
df_restaurant_4.to_csv('yelp_restaurant_formatted_4.csv',index=False)

**FIFTH AND LAST SUBSET:**

In [84]:
df_restaurant_5 = generate_restaurants_table(part_5)

resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one res

resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attribu

resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list

Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Fail

resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is em

resto_list is empty
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
There is more than one restaurant matched
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is 

resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
Failed retrieving one or more of the attributes
resto_list is empty
There is more than one restaurant matched
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes
resto_list is empty
resto_list is empty
resto_list is empty
resto_list is empty
Failed retrieving one or more of the attributes


In [85]:
df_restaurant_5

,ID,NAME,RATING,PRICE,PHONE,TRANSACTIONS,REVIEW_COUNT
0,Ftrk-Rx1i3okzxnQNuLcnw,Starbucks,2.5,$$,+12127422490,[delivery],96
1,6evDC62OwL1SewmR6ui6lw,Vivi Bubble Tea,3.5,$,+13477027899,"[delivery, pickup]",64
2,mnXrPqOVbX_06D9OAo4dLA,Atti,4.5,$$$$,+19179091122,"[pickup, delivery]",80
3,nfaS-7o4ZrVdsdqhZHnksQ,Alamo Drafthouse Cinema Downtown Brooklyn,3.5,$$,+17185132547,"[pickup, delivery]",835
4,_fTRTishY3G5dLPLYGCVAw,Tipsy Shanghai,4.0,$$,+12124666488,"[pickup, delivery]",53
...,...,...,...,...,...,...,...
265,VaE2VExpWLfLsgir8iCZ5w,Bella Vita II,3.5,$$,+17189838344,[],8
266,cfdljBG_M1Gtlou9_q0eWA,Osaka Fusion Sushi,4.5,$$,+13473744397,"[pickup, restaurant_reservation, delivery]",77
267,PT78C_bUMiYxzzsqO3kZ2Q,Caravan,4.0,$$$,+19173978435,"[pickup, delivery]",12
268,fUcjBGgmTlvxXn2azvAijw,Yumpling,4.5,$$,+17187131839,"[pickup, delivery]",405


In [86]:
# Save the restaurant details into CSV files
df_restaurant_5.to_csv('yelp_restaurant_formatted_5.csv',index=False)

In [87]:
len(df_restaurant_1) + len(df_restaurant_2) + len(df_restaurant_3) + len(df_restaurant_4) + len(df_restaurant_5)

8975

From our 20k+ restaurants, we only found a bit less than 9k with all the attributes we wanted on Yelp. This is pretty disappointing but still **about 1/3 of all NYC restaurants from the DOMHM data**.

We will now get an excerpt of the reviews from these businesses.

<h2> Step 4: Extract reviews from the restaurants </h2>

Unfortunately, Yelp API limits the number of reviews we can retrieve to 3 maximum per business. We will thus use these for EDA but probably not for modeling as it is not necessarily representative of the business as a whole.

<h3> 4.1. Define useful functions to query Yelp reviews API </h3>

In [81]:
# Create a function to get the reviews from one restaurant (unfortunately limited to 3 max)
def get_restaurant_review(api_key, restaurant_id):
    import json
    import requests
    business_path = BUSINESS_PATH + restaurant_id + "/reviews"
    url = API_HOST + business_path

    headers = {
        'Authorization': 'Bearer %s' % api_key,
    }

    response = requests.request('GET', url, headers=headers).json()
   
    review_text = ''
    counter = 0

    for review in response['reviews']: 
      review_text += review['text']  
      counter += 1

    res_review_df = pd.DataFrame([[restaurant_id, review_text]], columns = ['RESTAURANT_ID', 'REVIEWS'])
    
    return res_review_df

In [55]:
get_restaurant_review(API_KEY,'I6optu9x0JAf8oT5lqJ8qw')

,RESTAURANT_ID,REVIEWS
0,I6optu9x0JAf8oT5lqJ8qw,Stopped in for some shareables after a play. T...


In [82]:
# Create a function to generate a reviews dataframe from a restaurants dataframe
def generate_reviews_table(df):

  reviews_df = pd.DataFrame()
  
  for resto in df['ID']:
    try: 
      reviews_df = pd.concat([reviews_df,get_restaurant_review(API_KEY, resto)])
    except:
      continue
  
  reviews_df = reviews_df.reset_index(drop=True)
  
  return reviews_df

In [62]:
generate_reviews_table(df_restaurant_1[:3])

,RESTAURANT_ID,REVIEWS
0,LG9gS6YnT5NwootBWp7gPA,"The Morris Park Bake Shop is simply ""Classic"" ..."
1,AMxMPBkWi20dn_1BRalahA,The breakfast baconator w/ a sausage patty! No...
2,oo77ZO5cj9M2NRdTqEgHvQ,We were looking for an Irish pub in the Hell's...


<h3> 4.2. Apply these functions to our restaurants dataframes </h3>

Now that we have functions to query the API, we can reload our restaurants dataframe and apply these functions to them:

In [64]:
df_restaurant_1 = pd.read_csv('yelp_restaurant_formatted_1.csv')
df_restaurant_2 = pd.read_csv('yelp_restaurant_formatted_2.csv')
df_restaurant_3 = pd.read_csv('yelp_restaurant_formatted_3.csv')
df_restaurant_4 = pd.read_csv('yelp_restaurant_formatted_4.csv')
df_restaurant_5 = pd.read_csv('yelp_restaurant_formatted_5.csv')

As we did previously, we will have to comment/uncomment the values of API_KEY to switch between our various keys to avoid reaching the daily credits limit :)

**FIRST BATCH:**

In [68]:
df_reviews_1 = generate_reviews_table(df_restaurant_1)

In [69]:
df_reviews_1.to_csv('yelp_review_formatted_1.csv',index=False)

In [70]:
df_reviews_1

,RESTAURANT_ID,REVIEWS
0,LG9gS6YnT5NwootBWp7gPA,"The Morris Park Bake Shop is simply ""Classic"" ..."
1,AMxMPBkWi20dn_1BRalahA,The breakfast baconator w/ a sausage patty! No...
2,oo77ZO5cj9M2NRdTqEgHvQ,We were looking for an Irish pub in the Hell's...
3,Pe6MsH2DW0CXjvUtxUpI4A,Iabsolutely not...i used to go into this busin...
4,Q18r5FKi2qkQhtKZqWR8ww,We've ordered from these folks a number of tim...
...,...,...
3166,HhXXUpn4HlSqgWudxIcitA,"Inexpensive, good food, fast delivery. 1/4 chi..."
3167,hT17fpGyG_UvsBE4QUeYsA,"Dolly S.\nNew Hyde Park, NY\n\ndo not buy from..."
3168,fxGpXRxFUDzlU3Cyszu4uQ,Amazing food with a groovy atmosphere and a fr...
3169,Gis29CeO78xoaaoIL6BRKw,Never did I EVER think I would be writing a re...


We didn't lose any restaurants while retrieving reviews which is great! However, we'll have to be careful not to rely too much on these reviews are some of them are truncated and we are capped at 3 reviews per restaurant by Yelp API.

**SECOND BATCH:**

In [74]:
df_reviews_2 = generate_reviews_table(df_restaurant_2)

In [75]:
df_reviews_2.to_csv('yelp_review_formatted_2.csv',index=False)

In [76]:
df_reviews_2

,RESTAURANT_ID,REVIEWS
0,0zxGc7hit7mFSk2o1PvIdQ,I came with a party of 9 on a Saturday for a b...
1,GkxvCwfCFaFY4UDvAkZ8dQ,I went to Dallas BBQ a couple of times and foo...
2,pfmAcS-g6SiNG0KlLvrqnA,"Gerardo, the manager at Eastex and Beauty in W..."
3,nsAU6F2vmdnphmfIa_RuWg,"I've been eyeing this place for years, so glad..."
4,1fVp9c5Vw04ig171a_juRw,Always reliable. Hot and sour soup is my barom...
...,...,...
2669,IhGROM0nyXSpw2Z2oV-6aA,I tried the Mango Pineapple.smoothie for the 1...
2670,Aprs5McFDpJCKTxWEJMlag,I ordered a salad and medium pizza. I wasnt ex...
2671,MP8Ey2ezsDCSowR-1X6hKg,This place will help you satisfy your dosa cra...
2672,wYgp-defqwJPhjC6Y_WKWg,"One thing for sure, two things for certain; I ..."


**THIRD BATCH:**

In [77]:
df_reviews_3 = generate_reviews_table(df_restaurant_3)

In [78]:
df_reviews_3.to_csv('yelp_review_formatted_3.csv',index=False)

In [79]:
df_reviews_3

,RESTAURANT_ID,REVIEWS
0,05MSMrudU5N7UgImzuRVtw,This is the spot! Hands down they make the bes...
1,LbohfoklOxK7mSMm96_wLw,Amazing staff and amazing coffee! It's always ...
2,FiKu5C03pBiAn5v5DEL9UQ,This was a great vegetarian place for dinner b...
3,hTrn_YVmi5Kk7eVjTkBk9A,This McDonald's is in Queens Center Mall's foo...
4,GLWltF2ol24L4ypz5eTMhw,The lady that took my order was super rude and...
...,...,...
1947,TGWtZ3AnRZtftqnuueoSCg,We were in the neighborhood and came across th...
1948,Z6UJiyRrgix9uIrzDcCMUg,Great location. Good customer service. I actu...
1949,UQD85Z7Hiq_1Q3Jl7Jx-HQ,Very convenient for staying overnight before f...
1950,TkUWzErO5TBKl1jQcoKTng,"This is the best pizzeria eva, that's right, e..."


**FOURTH BATCH:**

In [83]:
df_reviews_4 = generate_reviews_table(df_restaurant_4)

In [84]:
df_reviews_4.to_csv('yelp_review_formatted_4.csv',index=False)

In [85]:
df_reviews_4

,RESTAURANT_ID,REVIEWS
0,C6GOh4ebry81vuG_765VvA,Stopped in for some cheesecake. The baked good...
1,I6optu9x0JAf8oT5lqJ8qw,Stopped in for some shareables after a play. T...
2,bJUDGmgdiicJHTmPtblFRg,I love this place nice and clean and close to ...
3,kGnJoYkzL-6aMrvwgYQXAg,This is my local Chinese restaurant. I've been...
4,jNPYmziqQjIMP-1pOgSZdw,I'd rate Kape't Torta four and a half stars.\n...
...,...,...
903,rmcuPv8ldKO5A8jd9-Y9zg,"Cute French goodies, tasty French bread and pa..."
904,c5rP7Em_FarV9GBzpl-yNQ,This review is solely based off a mobile order...
905,5UmAZm8hvNwbClnSBdnKJg,We were in NY for the weekend and I saw that t...
906,KEgdfVEVVgJFnGFiLNPDzQ,I just found my new favorite pizza spot in the...


**FIFTH AND LAST BATCH:**

In [86]:
df_reviews_5 = generate_reviews_table(df_restaurant_5)

In [87]:
df_reviews_5.to_csv('yelp_review_formatted_5.csv',index=False)

In [88]:
df_reviews_5

,RESTAURANT_ID,REVIEWS
0,Ftrk-Rx1i3okzxnQNuLcnw,Great location close to Broad Street J train s...
1,6evDC62OwL1SewmR6ui6lw,One of my favorite places to have bubble tea. ...
2,mnXrPqOVbX_06D9OAo4dLA,HANDS DOWN BEST KBQQ EVER- I had the pleasure ...
3,nfaS-7o4ZrVdsdqhZHnksQ,I visited this place when they first opened an...
4,_fTRTishY3G5dLPLYGCVAw,I love Dimsum shumai here is very good \nPrice...
...,...,...
265,VaE2VExpWLfLsgir8iCZ5w,Very Bad.\n\nHad my 17 birthday here and had a...
266,cfdljBG_M1Gtlou9_q0eWA,Fresh!!!!! Very fine dining in the area. Grea...
267,PT78C_bUMiYxzzsqO3kZ2Q,We ordered catering for my wife' birthday and ...
268,fUcjBGgmTlvxXn2azvAijw,I've been singing my praise about Yumpling wit...


<h2> Step 5: Merge all the data (Yelp & NYC Open Data) </h2>

<h3> 5.1. Merge all Yelp restaurants dataframes </h3>

In [91]:
yelp_restaurants = pd.concat([df_restaurant_1, df_restaurant_2, df_restaurant_3, df_restaurant_4, df_restaurant_5],ignore_index=True)

In [92]:
yelp_restaurants

,ID,NAME,RATING,PRICE,PHONE,TRANSACTIONS,REVIEW_COUNT
0,LG9gS6YnT5NwootBWp7gPA,Morris Park Bake Shop,4.0,$$,17188924968,['pickup'],66
1,AMxMPBkWi20dn_1BRalahA,Wendy's,1.5,$,17182875005,['delivery'],61
2,oo77ZO5cj9M2NRdTqEgHvQ,D.J. Reynolds,3.5,$$,12122452912,['delivery'],129
3,Pe6MsH2DW0CXjvUtxUpI4A,Wilkens Fine Foods,3.5,$$,17184443838,"['pickup', 'delivery']",64
4,Q18r5FKi2qkQhtKZqWR8ww,Taste the Tropics USA,4.5,$,17188560821,"['delivery', 'pickup']",59
...,...,...,...,...,...,...,...
8970,VaE2VExpWLfLsgir8iCZ5w,Bella Vita II,3.5,$$,17189838344,[],8
8971,cfdljBG_M1Gtlou9_q0eWA,Osaka Fusion Sushi,4.5,$$,13473744397,"['pickup', 'restaurant_reservation', 'delivery']",77
8972,PT78C_bUMiYxzzsqO3kZ2Q,Caravan,4.0,$$$,19173978435,"['pickup', 'delivery']",12
8973,fUcjBGgmTlvxXn2azvAijw,Yumpling,4.5,$$,17187131839,"['pickup', 'delivery']",405


<h3> 5.2. Merge all reviews dataframes </h3>

In [93]:
yelp_reviews = pd.concat([df_reviews_1, df_reviews_2, df_reviews_3, df_reviews_4, df_reviews_5],ignore_index=True)

In [94]:
yelp_reviews

,RESTAURANT_ID,REVIEWS
0,LG9gS6YnT5NwootBWp7gPA,"The Morris Park Bake Shop is simply ""Classic"" ..."
1,AMxMPBkWi20dn_1BRalahA,The breakfast baconator w/ a sausage patty! No...
2,oo77ZO5cj9M2NRdTqEgHvQ,We were looking for an Irish pub in the Hell's...
3,Pe6MsH2DW0CXjvUtxUpI4A,Iabsolutely not...i used to go into this busin...
4,Q18r5FKi2qkQhtKZqWR8ww,We've ordered from these folks a number of tim...
...,...,...
8970,VaE2VExpWLfLsgir8iCZ5w,Very Bad.\n\nHad my 17 birthday here and had a...
8971,cfdljBG_M1Gtlou9_q0eWA,Fresh!!!!! Very fine dining in the area. Grea...
8972,PT78C_bUMiYxzzsqO3kZ2Q,We ordered catering for my wife' birthday and ...
8973,fUcjBGgmTlvxXn2azvAijw,I've been singing my praise about Yumpling wit...


<h3> 5.3. Merge Yelp data to have a dataframe with restaurants attributes and reviews </h3>

In [95]:
# Rename the columns of yelp_reviews to match our yelp_restaurants columns
yelp_reviews.columns = ['ID', 'REVIEWS']

In [97]:
# Join the 2 dataframes on the restaurant Yelp ID
yelp_restaurants_with_reviews = yelp_restaurants.join(yelp_reviews.set_index('ID'), on='ID')

In [98]:
# Check that everything worked well
len(yelp_restaurants_with_reviews)

8975

In [99]:
yelp_restaurants_with_reviews

,ID,NAME,RATING,PRICE,PHONE,TRANSACTIONS,REVIEW_COUNT,REVIEWS
0,LG9gS6YnT5NwootBWp7gPA,Morris Park Bake Shop,4.0,$$,17188924968,['pickup'],66,"The Morris Park Bake Shop is simply ""Classic"" ..."
1,AMxMPBkWi20dn_1BRalahA,Wendy's,1.5,$,17182875005,['delivery'],61,The breakfast baconator w/ a sausage patty! No...
2,oo77ZO5cj9M2NRdTqEgHvQ,D.J. Reynolds,3.5,$$,12122452912,['delivery'],129,We were looking for an Irish pub in the Hell's...
3,Pe6MsH2DW0CXjvUtxUpI4A,Wilkens Fine Foods,3.5,$$,17184443838,"['pickup', 'delivery']",64,Iabsolutely not...i used to go into this busin...
4,Q18r5FKi2qkQhtKZqWR8ww,Taste the Tropics USA,4.5,$,17188560821,"['delivery', 'pickup']",59,We've ordered from these folks a number of tim...
...,...,...,...,...,...,...,...,...
8970,VaE2VExpWLfLsgir8iCZ5w,Bella Vita II,3.5,$$,17189838344,[],8,Very Bad.\n\nHad my 17 birthday here and had a...
8971,cfdljBG_M1Gtlou9_q0eWA,Osaka Fusion Sushi,4.5,$$,13473744397,"['pickup', 'restaurant_reservation', 'delivery']",77,Fresh!!!!! Very fine dining in the area. Grea...
8972,PT78C_bUMiYxzzsqO3kZ2Q,Caravan,4.0,$$$,19173978435,"['pickup', 'delivery']",12,We ordered catering for my wife' birthday and ...
8973,fUcjBGgmTlvxXn2azvAijw,Yumpling,4.5,$$,17187131839,"['pickup', 'delivery']",405,I've been singing my praise about Yumpling wit...


<h3> 5.4. Merge unique_restaurants_formatted with yelp_restaurants_with_reviews </h3>

Now, let's merge everything together!

In [100]:
# First, we need to reformat the phone number in our yelp dataframe to match the format of the other dataframe
yelp_restaurants_with_reviews['PHONE'] = yelp_restaurants_with_reviews['PHONE'].apply(lambda x: '+'+str(x))

In [101]:
yelp_restaurants_with_reviews['PHONE'].head(3) # All good!

0    +17188924968
1    +17182875005
2    +12122452912
Name: PHONE, dtype: object

In [102]:
final_restaurants_with_reviews = unique_restaurants_formatted.join(yelp_restaurants_with_reviews.set_index('PHONE'), how= 'inner', on='PHONE')

In [104]:
final_restaurants_with_reviews.tail(4)

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude,ID,NAME,RATING,PRICE,TRANSACTIONS,REVIEW_COUNT,REVIEWS
25924,50143511,OSAKA FUSION SUSHI & BBQ,Brooklyn,11214.0,+13473744397,Japanese,2023-11-20,Violations were cited in the following area(s).,A,13.0,40.605234,-73.999327,cfdljBG_M1Gtlou9_q0eWA,Osaka Fusion Sushi,4.5,$$,"['pickup', 'restaurant_reservation', 'delivery']",77,Fresh!!!!! Very fine dining in the area. Grea...
25932,50143576,CARAVAN RESTAURANT & COFFEE SHOP,Staten Island,10305.0,+19173978435,Asian/Asian Fusion,2024-01-04,Violations were cited in the following area(s).,A,10.0,40.592171,-74.067987,PT78C_bUMiYxzzsqO3kZ2Q,Caravan,4.0,$$$,"['pickup', 'delivery']",12,We ordered catering for my wife' birthday and ...
25974,50144227,YUMPLING,Queens,11101.0,+17187131839,Chinese,2024-01-03,Violations were cited in the following area(s).,A,13.0,40.743257,-73.953912,fUcjBGgmTlvxXn2azvAijw,Yumpling,4.5,$$,"['pickup', 'delivery']",405,I've been singing my praise about Yumpling wit...
25997,50144551,DIN'S KITCHEN,Brooklyn,11220.0,+13479874596,Chinese,2023-12-26,Violations were cited in the following area(s).,A,7.0,40.641202,-74.007217,wfkHwMHUxzmN4YjBbjSd9A,Beef & Fish,3.0,$$,"['delivery', 'pickup']",35,This place's dry hot pot is fitted flavored fo...


In [106]:
# We have to reset the index to get something neat
final_restaurants_with_reviews = final_restaurants_with_reviews.reset_index(drop=True)

In [107]:
final_restaurants_with_reviews

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude,ID,NAME,RATING,PRICE,TRANSACTIONS,REVIEW_COUNT,REVIEWS
0,30075445,MORRIS PARK BAKE SHOP,Bronx,10462.0,+17188924968,Bakery Products/Desserts,2023-08-22,Violations were cited in the following area(s).,A,12.0,40.848231,-73.855972,LG9gS6YnT5NwootBWp7gPA,Morris Park Bake Shop,4.0,$$,['pickup'],66,"The Morris Park Bake Shop is simply ""Classic"" ..."
1,30112340,WENDY'S,Brooklyn,11225.0,+17182875005,Hamburgers,2023-07-21,Violations were cited in the following area(s).,A,9.0,40.662652,-73.962081,AMxMPBkWi20dn_1BRalahA,Wendy's,1.5,$,['delivery'],61,The breakfast baconator w/ a sausage patty! No...
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,10019.0,+12122452912,Irish,2023-04-23,Violations were cited in the following area(s).,A,10.0,40.767326,-73.984310,oo77ZO5cj9M2NRdTqEgHvQ,D.J. Reynolds,3.5,$$,['delivery'],129,We were looking for an Irish pub in the Hell's...
3,40356483,WILKEN'S FINE FOOD,Brooklyn,11234.0,+17184443838,Sandwiches,2022-08-19,Violations were cited in the following area(s).,A,2.0,40.620112,-73.906989,Pe6MsH2DW0CXjvUtxUpI4A,Wilkens Fine Foods,3.5,$$,"['pickup', 'delivery']",64,Iabsolutely not...i used to go into this busin...
4,40356731,TASTE THE TROPICS ICE CREAM,Brooklyn,11226.0,+17188560821,Frozen Desserts,2023-01-17,Violations were cited in the following area(s).,A,9.0,40.640795,-73.948488,Q18r5FKi2qkQhtKZqWR8ww,Taste the Tropics USA,4.5,$,"['delivery', 'pickup']",59,We've ordered from these folks a number of tim...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8970,50142637,CASA VERDE,Staten Island,10306.0,+17189838344,Italian,2023-10-03,Violations were cited in the following area(s).,A,7.0,40.572978,-74.136456,VaE2VExpWLfLsgir8iCZ5w,Bella Vita II,3.5,$$,[],8,Very Bad.\n\nHad my 17 birthday here and had a...
8971,50143511,OSAKA FUSION SUSHI & BBQ,Brooklyn,11214.0,+13473744397,Japanese,2023-11-20,Violations were cited in the following area(s).,A,13.0,40.605234,-73.999327,cfdljBG_M1Gtlou9_q0eWA,Osaka Fusion Sushi,4.5,$$,"['pickup', 'restaurant_reservation', 'delivery']",77,Fresh!!!!! Very fine dining in the area. Grea...
8972,50143576,CARAVAN RESTAURANT & COFFEE SHOP,Staten Island,10305.0,+19173978435,Asian/Asian Fusion,2024-01-04,Violations were cited in the following area(s).,A,10.0,40.592171,-74.067987,PT78C_bUMiYxzzsqO3kZ2Q,Caravan,4.0,$$$,"['pickup', 'delivery']",12,We ordered catering for my wife' birthday and ...
8973,50144227,YUMPLING,Queens,11101.0,+17187131839,Chinese,2024-01-03,Violations were cited in the following area(s).,A,13.0,40.743257,-73.953912,fUcjBGgmTlvxXn2azvAijw,Yumpling,4.5,$$,"['pickup', 'delivery']",405,I've been singing my praise about Yumpling wit...


In [108]:
# Save everything as a csv file
final_restaurants_with_reviews.to_csv('final_restaurants_with_reviews.csv',index=False)

In [109]:
check = pd.read_csv('final_restaurants_with_reviews.csv')

In [110]:
check

,CAMIS,DBA,BORO,ZIPCODE,PHONE,CUISINE DESCRIPTION,INSPECTION DATE,ACTION,GRADE,SCORE,Latitude,Longitude,ID,NAME,RATING,PRICE,TRANSACTIONS,REVIEW_COUNT,REVIEWS
0,30075445,MORRIS PARK BAKE SHOP,Bronx,10462.0,17188924968,Bakery Products/Desserts,2023-08-22,Violations were cited in the following area(s).,A,12.0,40.848231,-73.855972,LG9gS6YnT5NwootBWp7gPA,Morris Park Bake Shop,4.0,$$,['pickup'],66,"The Morris Park Bake Shop is simply ""Classic"" ..."
1,30112340,WENDY'S,Brooklyn,11225.0,17182875005,Hamburgers,2023-07-21,Violations were cited in the following area(s).,A,9.0,40.662652,-73.962081,AMxMPBkWi20dn_1BRalahA,Wendy's,1.5,$,['delivery'],61,The breakfast baconator w/ a sausage patty! No...
2,30191841,DJ REYNOLDS PUB AND RESTAURANT,Manhattan,10019.0,12122452912,Irish,2023-04-23,Violations were cited in the following area(s).,A,10.0,40.767326,-73.984310,oo77ZO5cj9M2NRdTqEgHvQ,D.J. Reynolds,3.5,$$,['delivery'],129,We were looking for an Irish pub in the Hell's...
3,40356483,WILKEN'S FINE FOOD,Brooklyn,11234.0,17184443838,Sandwiches,2022-08-19,Violations were cited in the following area(s).,A,2.0,40.620112,-73.906989,Pe6MsH2DW0CXjvUtxUpI4A,Wilkens Fine Foods,3.5,$$,"['pickup', 'delivery']",64,Iabsolutely not...i used to go into this busin...
4,40356731,TASTE THE TROPICS ICE CREAM,Brooklyn,11226.0,17188560821,Frozen Desserts,2023-01-17,Violations were cited in the following area(s).,A,9.0,40.640795,-73.948488,Q18r5FKi2qkQhtKZqWR8ww,Taste the Tropics USA,4.5,$,"['delivery', 'pickup']",59,We've ordered from these folks a number of tim...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8970,50142637,CASA VERDE,Staten Island,10306.0,17189838344,Italian,2023-10-03,Violations were cited in the following area(s).,A,7.0,40.572978,-74.136456,VaE2VExpWLfLsgir8iCZ5w,Bella Vita II,3.5,$$,[],8,Very Bad.\n\nHad my 17 birthday here and had a...
8971,50143511,OSAKA FUSION SUSHI & BBQ,Brooklyn,11214.0,13473744397,Japanese,2023-11-20,Violations were cited in the following area(s).,A,13.0,40.605234,-73.999327,cfdljBG_M1Gtlou9_q0eWA,Osaka Fusion Sushi,4.5,$$,"['pickup', 'restaurant_reservation', 'delivery']",77,Fresh!!!!! Very fine dining in the area. Grea...
8972,50143576,CARAVAN RESTAURANT & COFFEE SHOP,Staten Island,10305.0,19173978435,Asian/Asian Fusion,2024-01-04,Violations were cited in the following area(s).,A,10.0,40.592171,-74.067987,PT78C_bUMiYxzzsqO3kZ2Q,Caravan,4.0,$$$,"['pickup', 'delivery']",12,We ordered catering for my wife' birthday and ...
8973,50144227,YUMPLING,Queens,11101.0,17187131839,Chinese,2024-01-03,Violations were cited in the following area(s).,A,13.0,40.743257,-73.953912,fUcjBGgmTlvxXn2azvAijw,Yumpling,4.5,$$,"['pickup', 'delivery']",405,I've been singing my praise about Yumpling wit...
